<a href="https://colab.research.google.com/github/akansh12/-Deep-Learning-for-Computer-Vision/blob/main/Part_A_Akansh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
import numpy as np
import torchvision
import os
import torch.nn as nn
from torch.nn import functional as F

In [2]:
import matplotlib.pyplot as plt
import copy

### Qurstion-1

In [3]:
### HELPER FUNCTIONS
def findConv2dOutShape(H_in,W_in,conv,pool=2):
  kernel_size=conv.kernel_size
  stride=conv.stride
  padding=conv.padding
  dilation=conv.dilation

  H_out=np.floor((H_in+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
  W_out=np.floor((W_in+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)
  if pool:
    H_out/=pool
    W_out/=pool
  return int(H_out),int(W_out)

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = "cpu"

In [43]:
class conv_net(nn.Module):
  def __init__(self, params): #num_filters, filter_size, activation_functions,dense_neurons,num_outputs
    super(conv_net,self).__init__()
    c_in, h_in, w_in = params["input_shape"]
    num_filters = params["num_filters"]
    filter_size = params["filter_size"]
    self.act_function = params["activation_functions"]
    dense_neurons = params["dense_neurons"]
    num_output = params["num_output"]
    self.dropout = params["dropout"]


    ####Conv layers
    self.conv1 = nn.Conv2d(c_in, num_filters[0], kernel_size=filter_size[0])
    h,w = findConv2dOutShape(h_in, w_in, self.conv1)

    self.conv2 = nn.Conv2d(num_filters[0], num_filters[1], kernel_size=filter_size[1])
    h,w = findConv2dOutShape(h, w, self.conv2)

    self.conv3 = nn.Conv2d(num_filters[1], num_filters[2], kernel_size=filter_size[2])
    h,w = findConv2dOutShape(h, w, self.conv3)


    self.conv4 = nn.Conv2d(num_filters[2], num_filters[3], kernel_size=filter_size[3])
    h,w = findConv2dOutShape(h, w, self.conv4)


    self.conv5 = nn.Conv2d(num_filters[3], num_filters[4], kernel_size=filter_size[4])
    h,w = findConv2dOutShape(h, w, self.conv5)
    #FC layers
    self.num_flatten = h*w*num_filters[4]
    self.fc1 = nn.Linear(self.num_flatten, dense_neurons)
    self.fc2 = nn.Linear(dense_neurons, num_output)

  def forward(self,x):
    x = self.act_function[0](self.conv1(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[1](self.conv2(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[2](self.conv3(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[3](self.conv4(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.act_function[4](self.conv5(x))
    x = F.max_pool2d(x, 2, 2)
    #Flatening the layers
    x = x.view(-1, self.num_flatten)
    
    x = self.act_function[5](self.fc1(x))
    x = F.dropout(x, self.dropout, training= self.training)
    x = self.fc2(x)
    x = F.log_softmax(x, dim=1)
    return x   


In [44]:
params = {"input_shape": (3,224,224),
          "num_filters": [8,16,32,64,128],
          "filter_size":[3,3,3,3,3],
          "activation_functions": [nn.functional.relu]*6,
          "dense_neurons": 128,
          "num_output":10,
          "dropout": 0.2          
          }

In [47]:
model = conv_net(params)
model.to(device);

1. What is the total number of computations done by your network? (assume m filters in each layer of size k×k and n neurons in the dense layer)

- Ans: 

2. What is the total number of parameters in your network? (assume m filters in each layer of size k×k and n neurons in the dense layer)

- Ans: 

### Question 2

In [48]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import os
path2data_train="/content/drive/MyDrive/Nature_data/inaturalist_12K/train"
path2data_test = "/content/drive/MyDrive/Nature_data/inaturalist_12K/val"

In [67]:
from torchvision.transforms.transforms import RandomRotation
# DATA-Augmentations
train_transforms = transforms.Compose([
transforms.Resize((224,224)),
transforms.RandomRotation((-20,20)),
transforms.RandomHorizontalFlip(p=0.5),
transforms.RandomVerticalFlip(p=0.5),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([
transforms.Resize((224,224)),
transforms.ToTensor(),
transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [68]:
data = datasets.ImageFolder(path2data_train, train_transforms)
test_data = datasets.ImageFolder(path2data_test, test_transforms)

In [59]:
print(data.class_to_idx)

{'Amphibia': 0, 'Animalia': 1, 'Arachnida': 2, 'Aves': 3, 'Fungi': 4, 'Insecta': 5, 'Mammalia': 6, 'Mollusca': 7, 'Plantae': 8, 'Reptilia': 9}


In [60]:
### Spiliting the data into train-val
n_val = int(np.floor(0.1 * len(data)))
n_train = len(data) - n_val
train_ds, val_ds = random_split(data, [n_train, n_val])

In [69]:
print("Number of datapoints in train: ", len(train_ds))
print("Number of datapoints in val: ", len(val_ds))
print("Number of datapoints in test: ", len(test_data))

Number of datapoints in train:  9000
Number of datapoints in val:  999
Number of datapoints in test:  2000


In [70]:
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=64, shuffle=False)
test_dl = DataLoader(test_data, batch_size=64, shuffle=False)

In [71]:
#Loss function
loss_func = nn.NLLLoss(reduction="sum")
#optimizer
from torch import optim
opt = optim.Adam(model.parameters(), lr=1e-4)

In [72]:
### Helper functions
def metrics_b(out, y_true):
  pred = out.argmax(dim=1, keepdim=True)
  true_pred=pred.eq(y_true.view_as(pred)).sum().item()
  return true_pred

def loss_batch(loss_func, out, y_true, opt=None):
  loss = loss_func(out, y_true)
  metric_batch = metrics_b(out,y_true)
  if opt is not None:
    opt.zero_grad()
    loss.backward()
    opt.step()
  return loss.item(), metric_batch

def loss_epoch(model,loss_func,data_loader,sanity_check=False,opt=None):
  run_loss=0.0
  running_metric=0.0
  len_data=len(data_loader.dataset)
  for x, y in data_loader:
    x=x.to(device)
    y=y.to(device)
    output=model(x)
    loss_b,metric_b=loss_batch(loss_func, output, y, opt)
    run_loss+=loss_b

    if metric_b is not None:
      running_metric+=metric_b

    if sanity_check is True:
      break

  loss=run_loss/float(len_data)
  metric=running_metric/float(len_data)

  return loss, metric

def train_val(model, params):
  num_epochs=params["num_epochs"]
  loss_func=params["loss_func"]
  opt=params["optimizer"]
  train_dl=params["train_dl"]
  val_dl=params["val_dl"]
  sanity_check=params["sanity_check"]
  lr_scheduler=params["lr_scheduler"]
  path2weights=params["path2weights"]

  loss_hist={
    "train": [],
    "val": [],
    }

  metric_hist={
    "train": [],
    "val": [],
    }

  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss=float('inf')

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs- 1))
    model.train()
    train_loss,train_metric=loss_epoch(model,loss_func,train_dl,sanity_check,opt)
    loss_hist["train"].append(train_loss)
    metric_hist["train"].append(train_metric)

    model.eval()
    with torch.no_grad():
      val_loss,val_metric = loss_epoch(model,loss_func,val_dl,sanity_check)
      loss_hist["val"].append(val_loss)
      metric_hist["val"].append(val_metric)






In [76]:
train_params = {"num_epochs": 30,
                "loss_func": loss_func,
                "train_dl":train_dl ,
                "val_dl":val_dl, 
                "test_dl": test_dl,
                "path2weights": "./"

               }